In [ ]:
import urllib
from bs4 import BeautifulSoup
import os
import pickle
import re
from queue import Queue
from threading import Thread
import time

In [ ]:
def in_val_set(i):
    return i % 10 == 0

def in_test_set(i):
    return i % 10 in [1, 2]

def in_train_set(i):
    return i % 10 > 2

In [ ]:
start_phrases = ["years ago", "min read", "(reuters) - ", "min read*"]
end_phrases = ["our standards:the thomson reuters trust principles", "reporting by", "editing by", "reporting and writing by", "additional reporting", "writing by"]

bad = 0

# https://stackoverflow.com/questions/3472515/python-urllib2-urlopen-is-slow-need-a-better-way-to-read-several-urls

# Fetch
def fetch_parallel(articles):
    dset = Queue()
    print("started making threads")
    threads = [Thread(target=read_webpage, args = (article, dset)) for article in articles]
    print("finished making threads")
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    return dset

def fetch_parallel2(articles):
    dset = Queue()
    threads = []
    for article in articles:
        thread = Thread(target=read_webpage, args = (article, dset))
        threads.append(thread)
        thread.start()
    for t in threads:
        t.join()
    return dset


def read_webpage(article, queue):
    title = article["title"]
    if title[:6] == "TABLE-":
        return

    link = article["href"]
    start_index = -1
    try:
        with urllib.request.urlopen(link) as webpage:
            content = BeautifulSoup(webpage.read(), "html.parser")
            for unnecessary_content in content(["script", "style", "table"]):
                unnecessary_content.extract()    
            text = content.get_text()
            for phrase in start_phrases:
                if phrase in text.lower():
                    phrase_index = text.lower().index(phrase) + len(phrase)
                    start_index = max(start_index, phrase_index)
            end_index = 99999999
            for phrase in end_phrases:
                if phrase in text.lower():
                    phrase_index = text.lower().index(phrase)
                    if text[phrase_index - 1] == "(":
                        phrase_index -= 1
                    end_index = min(end_index, phrase_index)
            text = text[start_index: end_index]
            text = text.replace("\n", " ")
            title = title.replace("\n", " ")
            queue.put((title, text))
    except:
        global bad
        bad += 1
    
def generate_dset(name, condition):
    # Load data
    articles = []
    for ls in os.listdir('Reuters-full-data-set-master/data')[:5]:
        print("New set", ls)
        if ls.endswith('.pkl'):
            with open('Reuters-full-data-set-master/data/' + ls, 'rb') as f:
                data = pickle.load(f, encoding='latin1')
                new_articles = [article for index, article in enumerate(data) if condition(index)][:50]
                articles += new_articles
                print("total article count:", len(articles))
#                 print("bad so far", bad)
    print("LOADED ALL FILES!")
    with open(name + "_articles.pkl", "wb") as outfile:
        pickle.dump(articles, outfile, pickle.HIGHEST_PROTOCOL)
    print("done dumping")
    dset = fetch_parallel2(articles)
    write_data(dset, name)
    
    
def write_data(dset, name):
    print("writing data")
    with open(name + "_source.txt", "a") as file_source:
        with open(name + "_target.txt", "a") as file_target:
#             i = 0
            while not dset.empty():
                article = dset.get()
#                 if not i % 1000: print(i)
#                 print("START", article[1], "END")
                file_source.write(article[0] + "\n")
                file_target.write(article[1] + "\n")
    print("DONE!!!")
                
                            
    
    
    
    

def generate_dset_old(name, condition):
#     with open(name + "_source.txt", "a") as file_source:
#         with open(name + "_target.txt", "a") as file_target:

            # Load data
            for ls in os.listdir('Reuters-full-data-set-master/data')[:1]:
                print("New set", ls)
                if ls.endswith('.pkl'):
                    with open('Reuters-full-data-set-master/data/' + ls, 'rb') as f:
                        data = pickle.load(f, encoding='latin1')
                        data = [article for index, article in enumerate(data) if condition(index)][:50]
                        print("total article count:", len(data))
                        for article in data:
                            title = article["title"]
                            if title[:6] == "TABLE-":
                                continue
                            
                            link = article["href"]
                            start_index = -1
                            try:
                                time1 = time.time()
                                with urllib.request.urlopen(link) as webpage:
                                    time2 = time.time()
                                    content = BeautifulSoup(webpage.read(), "html.parser")
                                    time3 = time.time()
                                    for unnecessary_content in content(["script", "style", "table"]):
                                        unnecessary_content.extract()    
                                    text = content.get_text()
                                    for phrase in start_phrases:
                                        if phrase in text.lower():
                                            phrase_index = text.lower().index(phrase) + len(phrase)
                                            start_index = max(start_index, phrase_index)
                                    end_index = 99999999
                                    for phrase in end_phrases:
                                        if phrase in text.lower():
                                            phrase_index = text.lower().index(phrase)
                                            if text[phrase_index - 1] == "(":
                                                phrase_index -= 1
                                            end_index = min(end_index, phrase_index)
                                    text = text[start_index: end_index]
                                    text = text.replace("\n", " ")
                                    title = title.replace("\n", " ")
                                    print(time3 - time2, time2 - time1)
#                                     file_source.write(title + "\n")
#                                     file_target.write(text + "\n")
#                                     print("suceeded!!!")
                            except Exception as e:
                                pass
#                                 print("BAD ARTICLE!!!")
#                                 print(e)
#                                 print(link)
#     print("DONE!!!")

# Our Standards:The Thomson Reuters Trust Principles

In [ ]:
# 10% val, 20% test, 70% train
print(len(os.listdir('Reuters-full-data-set-master/data')))
# print("Our Standards:The Thomson Reuters Trust Principles".lower())
# print(end_phrases[0])

In [ ]:
# Web scrape -P1
start = time.time()
generate_dset("validation", in_val_set)
end = time.time()
print("Total time: ", end - start)
# generate_dset("train", in_train_set)

In [ ]:
# Web scrape -P2
start = time.time()
generate_dset("validation", in_val_set)
end = time.time()
print("Total time: ", end - start)
# generate_dset("train", in_train_set)

In [ ]:
start = time.time()
generate_dset_old("validation", in_val_set)
end = time.time()
print("Total time: ", end - start)

In [ ]:
# Make dataset

https://stackoverflow.com/questions/328356/extracting-text-from-html-file-using-python

In [ ]:
url1 = "http://www.reuters.com/article/companyNewsAndPR/idUSTP13157220070102"
url2 = "http://www.reuters.com/article/2011/09/09/indian-stocks-open-idUSI8E7J302G20110909"
url3 = "http://www.reuters.com/article/companyNewsAndPR/idUSL1136380220070111"
url4 = "http://www.reuters.com/article/india-cenbank-refinance-idUSB8N12101C20151015"
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
start_phrase = "years ago"
end_phrase = "Our Standards:The Thomson Reuters Trust Principles."

with urllib.request.urlopen(url3) as webpage:
    content = BeautifulSoup(webpage.read(), "html.parser")
    for script in content(["script", "style"]):
        script.extract()
    print(content)
#     text = content.get_text()
#     start_index = text.index(start_phrase) + len(start_phrase)
#     end_index = text.index(end_phrase)
#     text = text[start_index: end_index]
#     lines = (line.strip() for line in text.splitlines() if not line.strip() == "")
#     for i, line in enumerate(lines):
# #         print("Line " + str(i) + ": " + line)
#         tokens = tokenizer.tokenize(line)
#         for token in tokens:
#             print("\n")
#             print(token)
# #         print(tokens)
        
